In [ ]:
# 🟩 STEP 0: Upload dataset ZIP
from google.colab import files
uploaded = files.upload()

In [ ]:
# 🟩 STEP 1: Extract ZIP
!unzip -q "dataset" -d /content/

In [ ]:
# 🟩 STEP 2: Confirm extraction
!ls /content

In [ ]:
# 🟩 STEP 3: Dataset directories
import os
base_dir = '/content'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
assert os.path.exists(train_dir), "Train folder not found."
assert os.path.exists(test_dir), "Test folder not found."

In [ ]:
# 🟩 STEP 4: Verify class folders
train_classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
test_classes = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])

print("Train classes:", train_classes)
print("Test classes:", test_classes)


In [ ]:
# 🟩 STEP 5: Image preprocessing with nail detection
import cv2
import numpy as np

def detect_nail_bbox(img, margin=0.1):
    """Detects the nail bounding box coordinates without cropping full image."""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return 0, 0, img.shape[1], img.shape[0]  # fallback: full image
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    mx = int(w * margin)
    my = int(h * margin)
    x1 = max(0, x - mx)
    y1 = max(0, y - my)
    x2 = min(img.shape[1], x + w + mx)
    y2 = min(img.shape[0], y + h + my)
    return x1, y1, x2, y2

def enhance_nail_image_full(img):
    """Preprocess image: normalization only, no cropping."""
    img = np.array(img, dtype=np.uint8)
    img = img / 255.0  # gentle normalization
    return img



In [ ]:
# 🟩 STEP 6: Data generators with safe augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.02,
    height_shift_range=0.02,
    zoom_range=0.03,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=enhance_nail_image_full
)

test_datagen = ImageDataGenerator(
    preprocessing_function=enhance_nail_image_full
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# 🟩 STEP 7: Visualize 9 full images with detected nail square
import matplotlib.pyplot as plt

images, labels = next(train_data)
plt.figure(figsize=(12, 8))

for i in range(9):
    img = (images[i]*255).astype(np.uint8).copy()  # restore 0-255 for cv2
    x1, y1, x2, y2 = detect_nail_bbox(img)

    # Draw rectangle around nail
    cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    img = img / 255.0  # back to 0-1 for plt

    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    label_idx = np.argmax(labels[i])
    label_name = list(train_data.class_indices.keys())[label_idx]
    plt.title(label_name, fontsize=9)
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# 🟩 STEP 8: Display class labels with numeric indices
for class_name, class_index in train_data.class_indices.items():
    print(f"{class_index} → {class_name}")